In [ ]:
from google.colab import drive
drive.mount('/content/drive')

###Emotion and Gender Recognition

In [ ]:

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
# path = '/content/drive/MyDrive/er_model.h5'
model = load_model('/content/drive/MyDrive/er_model.h5')


wav File (don't run if you're planning to record audio)

In [ ]:
import librosa
x, sr = librosa.load('/content/drive/MyDrive/fe3c06d9dfd30e585df4d20e593bfc8e.wav' , res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
#update path

Recording audio 

In [ ]:
!pip install ffmpeg-python

In [ ]:
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

record/stop button

In [ ]:
AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording.audio, press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving"
  }
}

function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

In [ ]:
def rec_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  riff_chunk_size = len(output)-8
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)
  riff = output[:4] + bytes(b) + output[8:]
  sr, x = wav_read(io.BytesIO(riff))

  return x, sr

In [ ]:
x, sr = get_audio()

Extracting MFCCs from the audio file

In [ ]:
import numpy as np
sr = np.array(sr)
mfccs = np.mean(librosa.feature.mfcc(y=x, n_mfcc=25),axis=0)
feature = mfccs
ex = feature

In [ ]:
import pandas as pd
ex = pd.DataFrame(data=ex)
ex = ex.stack().to_frame().T

In [ ]:
ex

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215
,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,-1.309638,-3.072465,-5.005925,-7.636365,-14.806997,-21.660511,-21.457758,-21.169487,-16.586264,-8.149051,-3.219617,-1.930284,-2.103285,-1.989719,-2.396503,-3.172928,-3.605502,-4.258596,-5.307495,-6.390612,-6.630668,-8.012033,-9.610266,-9.95483,-10.241838,-10.243867,-9.591209,-9.072826,-6.843174,-5.010115,-3.335672,-2.21436,-1.793612,-2.488196,-4.372026,-8.791296,-12.077606,-11.222675,-11.491663,-11.544604,...,-5.27577,-6.119092,-6.228513,-6.616949,-7.066959,-10.746784,-16.896509,-16.135952,-7.35425,-3.000881,0.007468,0.96296,1.34007,0.314369,-0.901048,-2.695438,-3.944601,-3.442747,-1.36564,-0.104621,0.509263,1.259713,1.714285,2.117019,2.230387,2.01263,1.497342,0.810993,0.425675,0.199952,-0.081277,-0.64889,-1.376027,-2.217439,-2.258772,-1.905654,-1.928122,-2.266734,-1.749749,0.175965


Predicting the gender and emotion of the speaker

In [ ]:
twod = np.expand_dims(ex, axis=2)
mod = model.predict(twod,batch_size=32,verbose=1)
mod = mod.argmax(axis=1)
ex1 = mod.astype(int).flatten()
print(mod)

1/1 [==============================] - 0s 408ms/step
[8]


In [ ]:
k = mod[0]
print(k)

8


Key:

0 - female, angry \\
1 - female, calm \\
2 - female, fearful \\
3 - female, happy \\
4 - female, sad \\
5 - male, angry \\
6 - male, calm \\
7 - male, fearful \\
8 - male, happy \\
9 - male, sad \\



In [ ]:
import IPython.display as ipd
ipd.Audio('/content/drive/MyDrive/fe3c06d9dfd30e585df4d20e593bfc8e.wav')

###ASR and Translation

In [ ]:
!pip3 install SpeechRecognition

     |████████████████████████████████| 32.8 MB 116 kB/s 


In [ ]:
import speech_recognition as sr
r = sr.Recognizer()
ger = sr.AudioFile('/content/drive/MyDrive/fe3c06d9dfd30e585df4d20e593bfc8e.wav')
with ger as source:
  audio = r.record(source)
german_text = r.recognize_google(audio, language="de-DE")

In [ ]:
print(german_text)

man sollte eher fragen wie viele das Geheimnis verstanden habe


In [ ]:
!pip3 install goslate

In [ ]:
import goslate
gs = goslate.Goslate()
english_text = gs.translate(german_text, 'en')
print(gs.translate(german_text, 'en'))

You should rather ask how many have understood the secret
